# Accessibility to schools in rural areas

In [ ]:
# If using colab
# Takes around 2-3 min
# !pip install "UrbanAccessAnalyzer[osm,plot,h3] @ git+https://github.com/CityScope/UrbanAccessAnalyzer.git"
# !pip install matplotlib mapclassify folium
# !apt-get install -y osmium-tool


# Restart notebook after installing this if needed

In [ ]:
import os
from datetime import datetime, date, timedelta, time
import pandas as pd
import geopandas as gpd
import os

import osmnx as ox

import matplotlib.pyplot as plt

import UrbanAccessAnalyzer.isochrones as isochrones
import UrbanAccessAnalyzer.graph_processing as graph_processing
import UrbanAccessAnalyzer.osm as osm
import UrbanAccessAnalyzer.utils as utils
import UrbanAccessAnalyzer.h3_utils as h3_utils
import UrbanAccessAnalyzer.population as population
import UrbanAccessAnalyzer.poi_utils as poi_utils

import zipfile
import numpy as np

#### Results folder

In [ ]:
results_path = os.path.normpath("output")
os.makedirs(results_path,exist_ok=True)

## 1 Area of interest

Area of interest (aoi): Polygon to do the analisys

Option 1: Write the city name

In [ ]:
city_name = "Arevalo, España"
city_filename = utils.sanitize_filename(city_name)
city_results_path = os.path.join(results_path,city_filename)
os.makedirs(city_results_path,exist_ok=True)
aoi = utils.get_city_geometry(city_name)
geo_suggestions = utils.get_geographic_suggestions_from_string(city_name,user_agent="app")
geo_suggestions

Option 2: Load your own file (.gpkg or .shp)

In [ ]:
# aoi = gpd.read_file("")
# city_name = ""

Use UTM coords and creat aoi_download with a buffer of X meters

In [ ]:
aoi = gpd.GeoDataFrame(geometry=[aoi.union_all()],crs=aoi.crs) # Ensure there is only one polygon
aoi = aoi.to_crs(aoi.estimate_utm_crs()) # Convert to utm

aoi_download = aoi.buffer(5000) # Area to do streets and poi requests 
# It should be the maximum buffer we will use but there is the risk of downloading an area that is too large

Map of your area of interest and the download area (aoi_buffer)

In [ ]:
m=aoi_download.explore(color='green')
m=aoi.explore(m=m,color='red')
m

## 2 Street graph

### 2.1 Regionwise file and cropping

- Download best regionwise pbf file. (Covers a large area)

- Crop it to cover our area of interest and save it in .osm format

In [ ]:
poi_path = os.path.normpath(city_results_path+f"/schools.gpkg")
osm_xml_file = os.path.normpath(city_results_path+f"/streets.osm")
streets_graph_path = os.path.normpath(city_results_path+f"/streets.graphml")
streets_path = os.path.normpath(city_results_path+f"/streets.gpkg")
level_of_service_streets_path = os.path.normpath(city_results_path+f"/level_of_service_streets.gpkg")
population_results_path = os.path.normpath(city_results_path+f"/population.gpkg")

In [ ]:
# Select what type of street network you want to load
network_filter = osm.osmium_network_filter("walk+bike+primary")
# Download the region pbf file crop it by aoi and convert to osm format
osm.geofabrik_to_osm(
    osm_xml_file,
    input_file=results_path,
    aoi=aoi_download,
    osmium_filter_args=network_filter,
    overwrite=False
)

### 2.2 Load to osmnx

This way the street network is a networkx graph

In [ ]:
# Load
G = ox.graph_from_xml(osm_xml_file)
# Project geometry coordinates to UTM system to allow euclidean meassurements in meters (sorry americans)
G = ox.project_graph(G,to_crs=aoi.estimate_utm_crs())
# Save the graph in graphml format to avoid the slow loading process
ox.save_graphml(G,streets_graph_path)

### 2.3 Simplify graph

Edges with length smaler than X meters are deleted and its nodes merged

In [ ]:
min_edge_length = 30

G = graph_processing.simplify_graph(G,min_edge_length=min_edge_length,min_edge_separation=min_edge_length*2,undirected=True)
# Save the result in graphml format
ox.save_graphml(G,streets_graph_path)

street_edges = ox.graph_to_gdfs(G,nodes=False)
street_edges = street_edges.to_crs(aoi.crs)
street_edges.to_file(streets_path)

# 3 Points of interest

## 3.1 Data source

#### A) Overpass API query

In [ ]:
query = """
[out:xml] [timeout:25];
(
    node["amenity"="school"]( {{bbox}});
    way["amenity"="school"]( {{bbox}});
    relation["amenity"="school"]( {{bbox}});
);
(._;>;);
out body;
"""

In [ ]:
# If it fails execute again

poi = osm.overpass_api_query(query,aoi_download)
poi.geometry = poi.geometry.centroid
poi = poi.to_crs(aoi.crs)
poi.to_file(poi_path)
poi

#### B) Load your own file

In [ ]:
# poi = gpd.read_file("")
# poi = poi.to_crs(aoi.crs)
# poi = poi[poi.geometry.intersects(aoi_download.union_all())]
# poi

### Lets see everything on a map

In [ ]:
m=aoi_download.explore(color='green')
m=aoi.explore(m=m,color='red')
m=poi.explore(m=m,color='black')
m

### 3.2 Add Points of interest to graph

In [ ]:
G, osmids = graph_processing.add_points_to_graph(
    poi,
    G,
    max_dist=100+min_edge_length, # Maximum distance from point to graph edge to project the point
    min_edge_length=min_edge_length # Minimum edge length after adding the new nodes
)
poi['osmid'] = osmids # Add the ids of the nodes in the graph to points

## 4 Compute isochrones

### 4.1 Distance steps

In [ ]:
distance_steps = [1000,3000,20000]
level_of_services = ['walk','bike','bus']

### 4.2 Isochrones

In [ ]:
level_of_service_graph = isochrones.graph(
    G,
    poi,
    distance_steps, # If service_quality_col is None it could be a list of distances
    service_quality_col = None, # If all points have the same quality this could be None
    level_of_services = level_of_services, # could be None and it will set to the sorted unique values of the matrix
    min_edge_length = min_edge_length # Do not add new nodes if there will be an edge with less than this length
)
# Save edges as gpkg
level_of_service_nodes, level_of_service_edges = ox.graph_to_gdfs(level_of_service_graph)
level_of_service_edges.to_file(level_of_service_streets_path)

#### Lets visualize the results on a map

### 4.3 Convert to H3

In [ ]:
h3_resolution = 10

In [ ]:
ls_h3_df = h3_utils.from_gdf(
    level_of_service_edges,
    resolution=h3_resolution,
    value_column='level_of_service',
    value_order=level_of_services,
    buffer=10
)

ls_h3_df

In [ ]:
m = None

# m = street_edges.explore(
#     m = m,
#     color='black'
# )

m = h3_utils.h3_to_gdf(ls_h3_df).explore(
    m=m,
    column='level_of_service',
    cmap="RdYlGn_r",
    style_kwds={
        "color": None,   # no border color
        "weight": 0,     # border width
        "fillOpacity": 0.3,  # optional fill transparency
    }
)

m = level_of_service_edges.explore(
    m=m,
    column='level_of_service',
    cmap="RdYlGn_r",
    style_kwds={
        "weight": 3,        # thickness of the polygon/line border
    }
)

m = poi[[
    "name",
    "geometry"
]].explore(
    m=m,
    color="black",
    style_kwds={
        "color": "black",       # Border color
        "weight": 1,            # Border thickness
        "opacity": 1.0,         # Border opacity
        "fillOpacity": 1,
        "radius": 4,
    },
)
m.save(city_results_path + "/level_of_service.html")
m


## 5 Population

### 5.1 Download Worldpop tif file

- One file for every country
- 100m pixel size
- tif format
- available from 2000 to 2030
- gender and age

In [ ]:
population_file = population.download_worldpop_population(
    aoi_download,
    2025,
    folder=results_path,
    resolution="100m",
    dataset="age_structures",
    subset="U18"
)

In [ ]:
zip_path = population_file

# Extract to the same directory as the zip file
extract_dir = os.path.splitext(zip_path)[0]
os.makedirs(extract_dir, exist_ok=True)

# Decompress the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Find the .tif file that contains '_T_'
for file_name in os.listdir(extract_dir):
    if file_name.lower().endswith('.tif') and '_T_' in file_name:
        population_file = os.path.join(extract_dir, file_name)
        break
else:
    raise FileNotFoundError("No .tif file containing '_T_' found in the zip archive.")

In [ ]:
pop_h3_df = h3_utils.from_raster(population_file,aoi=aoi_download,resolution=h3_resolution)
pop_h3_df = pop_h3_df.rename(columns={'value':'population'})

### 5.2 Assign level of service to each population cell

In [ ]:
results_h3_df = ls_h3_df.merge(pop_h3_df,left_index=True,right_index=True,how='outer')
results_h3_df = h3_utils.h3_to_gdf(results_h3_df).to_crs(aoi.crs)
results_h3_df = results_h3_df[results_h3_df.intersects(aoi.union_all())]
results_h3_df.to_file(population_results_path)
results_h3_df

In [ ]:
m = None

# m = street_edges.explore(
#     m = m,
#     color='black'
# )

m = level_of_service_edges.explore(
    m=m,
    column='level_of_service',
    cmap="RdYlGn_r",
)

m = poi[[
    #"name",
    "geometry"
]].explore(
    m=m,
    color="green",
    style_kwds={
        "color": "black",       # Border color
        "weight": 1,            # Border thickness
        "opacity": 1.0,         # Border opacity
        "fillOpacity": 1,
        "radius": 4,
    },
)

pop_gdf_points = results_h3_df.copy()
pop_gdf_points.geometry = pop_gdf_points.geometry.centroid
pop_gdf_points = pop_gdf_points.dropna(subset=['population'])
pop_gdf_points = pop_gdf_points[pop_gdf_points['population'] > 1]
m=pop_gdf_points.explore(
    m=m,
    column="level_of_service",            # color by service level
    cmap="RdYlGn_r",
    legend=True,
    tooltip=["population", "level_of_service"],
    marker_type="circle_marker",
    style_kwds={
        # define a dynamic radius for each feature
        "style_function": lambda x: {
            "radius": (np.log1p(x["properties"]["population"]) /
                       np.log1p(pop_gdf_points["population"].max()) * 5),
            "color": "black",      # border color
            "weight": 1,           # border thickness
            "opacity": 1.0,        # border opacity
            "fillOpacity": 1.0,    # fill opacity
        },
    },
)
m.save(city_results_path + "/population.html")
m

Important files:

- streets.gpkg Has the street geometry as lines (all streets)
- level_of_service_streets.gpkg Has the street geometry as lines with the level of service (only streets with level of service)
- population.gpkg Is a grid with population and level of service